In [34]:
import pandas as pd
import numpy as np
import re #Regex lib
from forex_python.converter import CurrencyRates #currencies lib
from datetime import datetime
import matplotlib

### Read in the dataframes and create a dict {(year, dataframe)}


In [35]:
# Read data frames
data_frames_dict = {}

data_frames_dict[2018] = pd.read_csv('datasets/2018.csv', encoding = "ISO-8859-1").filter(items=['Country', 'LanguageWorkedWith', 'LanguageDesireNextYear']).dropna(thresh=3) 
data_frames_dict[2019] = pd.read_csv('datasets/2019.csv', encoding = "ISO-8859-1").filter (items=['Country',  'LanguageWorkedWith', 'LanguageDesireNextYear']).dropna(thresh=3) 
data_frames_dict[2020] = pd.read_csv('datasets/2020.csv', encoding = "ISO-8859-1").filter (items=['Country',  'LanguageWorkedWith', 'LanguageDesireNextYear']).dropna(thresh=3) 
data_frames_dict[2021] = pd.read_csv('datasets/2021.csv', encoding = "ISO-8859-1").filter (items=['Country',  'LanguageHaveWorkedWith', 'LanguageWantToWorkWith']).dropna(thresh=3) 


/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### obtain developer types 


In [36]:
for year, df in data_frames_dict.items():
    df.columns = ['Country',  'LanguageWorkedWith', 'LanguageDesireNextYear']
    df.LanguageWorkedWith = df.LanguageWorkedWith.apply(lambda s: s.split(';'))
    df.LanguageDesireNextYear = df.LanguageDesireNextYear.apply(lambda s: s.split(';'))
    
    #LanguageWorkedWith
    
    df_LanguageWorkedWith = df.filter(items = ['Country',  'LanguageWorkedWith'])
    df_LanguageWorkedWith = df_LanguageWorkedWith.explode('LanguageWorkedWith')
    df_LanguageWorkedWith['LanguageWorkedWithCount'] = pd.Series(np.ones(len(df_LanguageWorkedWith.LanguageWorkedWith)), index=df_LanguageWorkedWith.index)  
    df_LanguageWorkedWith = df_LanguageWorkedWith.groupby(by=['Country','LanguageWorkedWith'] , as_index=False).sum()
    df_LanguageWorkedWith.to_csv(path_or_buf = 'processed_data/languages/{}-LanguageWorkedWith_count_per_country.csv'.format(year), index=False)
    
    # LanguageDesireNextYear
    df_LanguageDesireNextYear = df.filter(items = ['Country',  'LanguageDesireNextYear'])
    df_LanguageDesireNextYear = df_LanguageDesireNextYear.explode('LanguageDesireNextYear')
    df_LanguageDesireNextYear['LanguageDesireNextYearCount'] = pd.Series(np.ones(len(df_LanguageDesireNextYear.LanguageDesireNextYear)), index=df_LanguageDesireNextYear.index)  
    df_LanguageDesireNextYear = df_LanguageDesireNextYear.groupby(by=['Country','LanguageDesireNextYear'] , as_index=False).sum()
    df_LanguageDesireNextYear.to_csv(path_or_buf = 'processed_data/languages/{}-LanguageDesireNextYear_count_per_country.csv'.format(year), index=False)



        


In [37]:
df_LanguageDesireNextYear


,Country,LanguageDesireNextYear,LanguageDesireNextYearCount
0,Afghanistan,APL,4.0
1,Afghanistan,Assembly,6.0
2,Afghanistan,Bash/Shell,7.0
3,Afghanistan,C,9.0
4,Afghanistan,C#,12.0
...,...,...,...
4991,Zimbabwe,Ruby,2.0
4992,Zimbabwe,Rust,4.0
4993,Zimbabwe,SQL,10.0
4994,Zimbabwe,Swift,5.0


In [38]:
df_LanguageWorkedWith

,Country,LanguageWorkedWith,LanguageWorkedWithCount
0,Afghanistan,APL,7.0
1,Afghanistan,Assembly,9.0
2,Afghanistan,Bash/Shell,12.0
3,Afghanistan,C,11.0
4,Afghanistan,C#,15.0
...,...,...,...
4839,Zimbabwe,SQL,10.0
4840,Zimbabwe,Scala,1.0
4841,Zimbabwe,Swift,1.0
4842,Zimbabwe,TypeScript,5.0
